<a href="https://colab.research.google.com/github/sandrakaku/ml0930/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
# ((訓練圖片, 訓練答案), (側視圖片, 測試答案))
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
import random 
import matplotlib.pyplot as plt
import pandas as pd
idx = random.randint(0, 59999)
plt.imshow(x_train[idx], cmap="gray") #加_r r=reverse 顏色反轉
pd.DataFrame(x_train[idx]) 

In [ ]:
from tensorflow.keras.models import Sequential #model大廈 Sequential樓層 的概念
from tensorflow.keras.layers import Dense #全連接(每一層都跟下一層有關係)
layers = [
    # 784 * 128 + 128(bias)
    Dense(128, activation="relu", input_dim=784), #28*28=784
    # 128 * 10 + 10(bias)
    Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
# x:scaling 0-255 -> 0-1 框架(tf)隨機係數的關係
# y:one-hot encoding 十個答案
from tensorflow.keras.utils import to_categorical

x_train.reshape(-1, 784) #-1可以代表不知道請電腦自己代
x_train_norm = x_train.reshape(-1, 784) / 255 #框架的關係
x_test_norm = x_test.reshape(-1, 784) / 255